In [109]:
import boto3
from datetime import date
import time
import uuid
from credentials import *
import unittest

## Connect to DynamoDB

Initially, we connect to our local database we started as "dynamodb" container:

In [110]:
client = boto3.client('dynamodb', 
  endpoint_url='http://dynamo:8000', 
  region_name='eu-central-1',
  aws_access_key_id=aws_access_key_id,
  aws_secret_access_key=aws_secret_access_key
)
client.list_tables()

{'TableNames': ['twitter'],
 'ResponseMetadata': {'RequestId': 'b8b6db63-91a6-4bed-892b-238518f07504',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Fri, 12 Jan 2024 12:00:45 GMT',
   'x-amzn-requestid': 'b8b6db63-91a6-4bed-892b-238518f07504',
   'content-type': 'application/x-amz-json-1.0',
   'x-amz-crc32': '2569046870',
   'content-length': '26',
   'server': 'Jetty(11.0.17)'},
  'RetryAttempts': 0}}

## Create Table

Let's setup the table with our attributes and indexes:

In [111]:
def create_app_table():
    return client.create_table(
        TableName="twitter",
        KeySchema=[
            {"AttributeName": "PK", "KeyType": "HASH"},
            {"AttributeName": "SK", "KeyType": "RANGE"},
        ],
        AttributeDefinitions=[
            {"AttributeName": "PK", "AttributeType": "S"},
            {"AttributeName": "SK", "AttributeType": "S"},
        ],
        ProvisionedThroughput={"ReadCapacityUnits": 10, "WriteCapacityUnits": 10},
    )

In [112]:
def delete_app_table():
    try:
        return client.delete_table(TableName="twitter")
    except:
        return False

In [113]:
delete_app_table()
create_app_table()

{'TableDescription': {'AttributeDefinitions': [{'AttributeName': 'PK',
    'AttributeType': 'S'},
   {'AttributeName': 'SK', 'AttributeType': 'S'}],
  'TableName': 'twitter',
  'KeySchema': [{'AttributeName': 'PK', 'KeyType': 'HASH'},
   {'AttributeName': 'SK', 'KeyType': 'RANGE'}],
  'TableStatus': 'ACTIVE',
  'CreationDateTime': datetime.datetime(2024, 1, 12, 12, 0, 46, 64000, tzinfo=tzlocal()),
  'ProvisionedThroughput': {'LastIncreaseDateTime': datetime.datetime(1970, 1, 1, 0, 0, tzinfo=tzlocal()),
   'LastDecreaseDateTime': datetime.datetime(1970, 1, 1, 0, 0, tzinfo=tzlocal()),
   'NumberOfDecreasesToday': 0,
   'ReadCapacityUnits': 10,
   'WriteCapacityUnits': 10},
  'TableSizeBytes': 0,
  'ItemCount': 0,
  'TableArn': 'arn:aws:dynamodb:ddblocal:000000000000:table/twitter',
  'DeletionProtectionEnabled': False},
 'ResponseMetadata': {'RequestId': '1480fc81-372d-4587-882b-9a54c894bbb0',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Fri, 12 Jan 2024 12:00:46 GMT',
   'x-amzn-

## Helpers

To setup some other keys later on, we create a function to extract the id from a key:

In [114]:
def parse_id_from_key(key):
    return key.split("#")[-1]

parse_id_from_key("CHAN#7")

'7'

## Users

We create and save our users:

In [115]:
uuid_entry1 = str(uuid.uuid4())
user_martin = {
    'PK': {'S': f"USER#{uuid_entry1}"},
    'SK': {'S': f"USER#{uuid_entry1}"},
    'Name': {'S': 'Martin Marsal'},
}

uuid_entry2 = str(uuid.uuid4())
user_christian = {
    'PK': {'S': f"USER#{uuid_entry2}"},
    'SK': {'S': f"USER#{uuid_entry2}"},
    'Name': {'S': 'Christian Diegmann'},
}

uuid_entry3 = str(uuid.uuid4())
user_robin = {
    'PK': {'S': f"USER#{uuid_entry3}"},
    'SK': {'S': f"USER#{uuid_entry3}"},
    'Name': {'S': 'Robin Schüle'},
}

def save_user(user):
    return client.put_item(
        TableName="twitter",
        Item=user
    )

# AP6
def find_user(primaryKey, sortKey):
    item = client.get_item(
        TableName="twitter",
        Key={
          'PK': { 'S': primaryKey },
          'SK': { 'S': sortKey }
        }
    )
    return item['Item'] if 'Item' in item else False

In [116]:
save_user(user_martin)
find_user(user_martin['PK']['S'], user_martin['SK']['S'])

{'SK': {'S': 'USER#6cca7afc-e1ec-4eca-b0eb-175480e275b5'},
 'PK': {'S': 'USER#6cca7afc-e1ec-4eca-b0eb-175480e275b5'},
 'Name': {'S': 'Martin Marsal'}}

In [117]:
save_user(user_christian)
find_user(user_christian['PK']['S'], user_christian['SK']['S'])

{'SK': {'S': 'USER#0c074896-cd18-4e0d-8df9-cebb9bacdfa5'},
 'PK': {'S': 'USER#0c074896-cd18-4e0d-8df9-cebb9bacdfa5'},
 'Name': {'S': 'Christian Diegmann'}}

In [118]:
save_user(user_robin)
find_user(user_robin['PK']['S'], user_robin['SK']['S'])

{'SK': {'S': 'USER#a7324fea-4a38-45a3-82ad-27c7bf0ea941'},
 'PK': {'S': 'USER#a7324fea-4a38-45a3-82ad-27c7bf0ea941'},
 'Name': {'S': 'Robin Schüle'}}

In [119]:
def add_to_followers(primaryKey, follower):
    response = client.put_item(
        TableName="twitter",
        Item={
            'PK': {'S': primaryKey},
            'SK': {'S': f"FOLLOWER#{parse_id_from_key(follower['PK']['S'])}"},
            'Name': {'S': follower['Name']['S']},
        },
        )
    if response['ResponseMetadata']['HTTPStatusCode'] == 200:
        return client.get_item(
            TableName='twitter',
            Key={
                'PK': {'S': primaryKey},
                'SK': {'S': f"FOLLOWER#{parse_id_from_key(follower['PK']['S'])}"},
            }
        )

In [120]:
add_to_followers(user_martin['PK']['S'], user_christian)

{'Item': {'SK': {'S': 'FOLLOWER#0c074896-cd18-4e0d-8df9-cebb9bacdfa5'},
  'PK': {'S': 'USER#6cca7afc-e1ec-4eca-b0eb-175480e275b5'},
  'Name': {'S': 'Christian Diegmann'}},
 'ResponseMetadata': {'RequestId': '2f91f3e9-91d1-447b-976f-f4694ae6e483',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Fri, 12 Jan 2024 12:00:47 GMT',
   'x-amzn-requestid': '2f91f3e9-91d1-447b-976f-f4694ae6e483',
   'content-type': 'application/x-amz-json-1.0',
   'x-amz-crc32': '2827975585',
   'content-length': '158',
   'server': 'Jetty(11.0.17)'},
  'RetryAttempts': 0}}

In [121]:
add_to_followers(user_christian['PK']['S'], user_robin)

{'Item': {'SK': {'S': 'FOLLOWER#a7324fea-4a38-45a3-82ad-27c7bf0ea941'},
  'PK': {'S': 'USER#0c074896-cd18-4e0d-8df9-cebb9bacdfa5'},
  'Name': {'S': 'Robin Schüle'}},
 'ResponseMetadata': {'RequestId': '5dd2b16f-aa38-4d91-97af-442d6695d4c3',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Fri, 12 Jan 2024 12:00:47 GMT',
   'x-amzn-requestid': '5dd2b16f-aa38-4d91-97af-442d6695d4c3',
   'content-type': 'application/x-amz-json-1.0',
   'x-amz-crc32': '2434205436',
   'content-length': '153',
   'server': 'Jetty(11.0.17)'},
  'RetryAttempts': 0}}

In [122]:
add_to_followers(user_robin['PK']['S'], user_martin)

{'Item': {'SK': {'S': 'FOLLOWER#6cca7afc-e1ec-4eca-b0eb-175480e275b5'},
  'PK': {'S': 'USER#a7324fea-4a38-45a3-82ad-27c7bf0ea941'},
  'Name': {'S': 'Martin Marsal'}},
 'ResponseMetadata': {'RequestId': 'b4f7f34e-34a6-4cb2-8a75-13492b2dceaf',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Fri, 12 Jan 2024 12:00:47 GMT',
   'x-amzn-requestid': 'b4f7f34e-34a6-4cb2-8a75-13492b2dceaf',
   'content-type': 'application/x-amz-json-1.0',
   'x-amz-crc32': '3527830353',
   'content-length': '153',
   'server': 'Jetty(11.0.17)'},
  'RetryAttempts': 0}}

## 1st access pattern: Post a tweet

In [123]:
uuid_entry4 = str(uuid.uuid4())
timestamp = int(time.time())
tweet = {'id': {'S': f"{uuid_entry4}"}, 'text': {'S': 'Moin, moin.'}, 'likes': {'N': '0'}, 'CreatedAt': { 'N': str(timestamp)}}
uuid_entry5 = str(uuid.uuid4())
timestamp = int(time.time())
tweet_2 = {'id': {'S': f"{uuid_entry5}"}, 'text': {'S': 'MongoDB ist super!'}, 'likes': {'N': '0'}, 'CreatedAt': { 'N': str(timestamp)}}
newMartin = find_user(user_martin['PK']['S'], user_martin['SK']['S'])

def post_tweet(primaryKey, tweet):
    client.put_item(
        TableName='twitter',
        Item={
            'PK': {'S': primaryKey},
            'SK': {'S': f"TWEET#{tweet['id']['S']}"},
            'id': {'S': tweet['id']['S']},
            'text': {'S': tweet['text']['S']},
            'likes': {'N': tweet['likes']['N']},
            'CreatedAt': { 'N': tweet['CreatedAt']['N']}
        },
    )
    item = client.query(
        TableName="twitter",
        KeyConditionExpression='PK = :pk AND begins_with(SK, :follower)',
        ExpressionAttributeValues={
            ':pk': { 'S': primaryKey },
            ':follower': { 'S': 'FOLLOWER#' }
        },
    )
    followers = item['Items']
    for follower in followers:
        response2 = client.put_item(
            TableName='twitter',
            Item={
                'PK': {'S': f"USER#{parse_id_from_key(follower['SK']['S'])}"},
                'SK': {'S': f"TIMELINE#{tweet['id']['S']}"},
                'tweetId': {'S': tweet['id']['S']},
                'text': {'S': tweet['text']['S']},
                'likes': {'N': tweet['likes']['N']},
                'CreatedAt': { 'N': tweet['CreatedAt']['N']}
            },
        )
    if response2['ResponseMetadata']['HTTPStatusCode'] == 200:
        return client.query(
            TableName="twitter",
            KeyConditionExpression='PK = :pk AND begins_with(SK, :timeline)',
            ExpressionAttributeValues={
                ':pk': { 'S': f"USER#{parse_id_from_key(user_christian['PK']['S'])}" },
                ':timeline': { 'S': 'TIMELINE#' }
            },
        )

In [16]:
post_tweet(user_martin['PK']['S'], tweet)

{'Items': [{'SK': {'S': 'TIMELINE#5a03ab7c-daf3-4626-8fd9-00671049ca89'},
   'PK': {'S': 'USER#96b97410-a549-430a-b6a4-048c1152f103'},
   'text': {'S': 'Moin, moin.'},
   'tweetId': {'S': '5a03ab7c-daf3-4626-8fd9-00671049ca89'},
   'likes': {'N': '0'},
   'CreatedAt': {'N': '1705014707'}}],
 'Count': 1,
 'ScannedCount': 1,
 'ResponseMetadata': {'RequestId': '8d00aa9d-df93-43c4-be5d-5754f3cbc4aa',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Thu, 11 Jan 2024 23:11:47 GMT',
   'x-amzn-requestid': '8d00aa9d-df93-43c4-be5d-5754f3cbc4aa',
   'content-type': 'application/x-amz-json-1.0',
   'x-amz-crc32': '3456900121',
   'content-length': '285',
   'server': 'Jetty(11.0.17)'},
  'RetryAttempts': 0}}

In [17]:
post_tweet(user_martin['PK']['S'], tweet_2)

{'Items': [{'SK': {'S': 'TIMELINE#36cd887d-e151-48be-a36c-2c239997d726'},
   'PK': {'S': 'USER#96b97410-a549-430a-b6a4-048c1152f103'},
   'text': {'S': 'MongoDB ist super!'},
   'tweetId': {'S': '36cd887d-e151-48be-a36c-2c239997d726'},
   'likes': {'N': '0'},
   'CreatedAt': {'N': '1705014707'}},
  {'SK': {'S': 'TIMELINE#5a03ab7c-daf3-4626-8fd9-00671049ca89'},
   'PK': {'S': 'USER#96b97410-a549-430a-b6a4-048c1152f103'},
   'text': {'S': 'Moin, moin.'},
   'tweetId': {'S': '5a03ab7c-daf3-4626-8fd9-00671049ca89'},
   'likes': {'N': '0'},
   'CreatedAt': {'N': '1705014707'}}],
 'Count': 2,
 'ScannedCount': 2,
 'ResponseMetadata': {'RequestId': '827fab4c-2edc-4e34-a402-343057f26f23',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Thu, 11 Jan 2024 23:11:47 GMT',
   'x-amzn-requestid': '827fab4c-2edc-4e34-a402-343057f26f23',
   'content-type': 'application/x-amz-json-1.0',
   'x-amz-crc32': '2758427574',
   'content-length': '539',
   'server': 'Jetty(11.0.17)'},
  'RetryAttempts': 0}}

In [18]:
class TestPostTweet(unittest.TestCase):
    def test_post_tweet_enriched_timeline(self):
        christian_timeline = client.query(
            TableName="twitter",
            KeyConditionExpression='PK = :pk AND begins_with(SK, :timeline)',
            ExpressionAttributeValues={
                ':pk': { 'S': user_christian['PK']['S'] },
                ':timeline': { 'S': 'TIMELINE#' }
            },
        )
        timeline_length = len(christian_timeline['Items'])
        self.assertEqual(timeline_length, 2)

    def test_post_tweet_empty_timeline(self):
        robin_timeline = client.query(
            TableName="twitter",
            KeyConditionExpression='PK = :pk AND begins_with(SK, :timeline)',
            ExpressionAttributeValues={
                ':pk': { 'S': user_robin['PK']['S'] },
                ':timeline': { 'S': 'TIMELINE#' }
            },
        )
        timeline_length = len(robin_timeline['Items'])
        self.assertEqual(timeline_length, 0)

In [19]:
if __name__ == '__main__':
    suite = unittest.TestLoader().loadTestsFromTestCase(TestPostTweet)
    unittest.TextTestRunner(verbosity=2).run(suite)

test_post_tweet_empty_timeline (__main__.TestPostTweet.test_post_tweet_empty_timeline) ... ok
test_post_tweet_enriched_timeline (__main__.TestPostTweet.test_post_tweet_enriched_timeline) ... ok

----------------------------------------------------------------------
Ran 2 tests in 0.017s

OK


## 2nd access pattern: Write a reply

In [20]:
uuid_entry6 = str(uuid.uuid4())
reply = {'id': {'S': f"{uuid_entry6}"}, 'text': {'S': 'Hallo zurück.'}, 'likes': {'N': '0'}, 'CreatedAt': { 'N': str(timestamp)}, 'tweetId': {'S': f"{parse_id_from_key(tweet['id']['S'])}"},}

def post_reply(author, user, tweet, reply):
    client.put_item(
        TableName='twitter',
        Item={
            'PK': {'S': f"USER#{parse_id_from_key(author['PK']['S'])}"},
            'SK': {'S': f"TWEET#{tweet['id']['S']}#{reply['id']['S']}"},
            'id': {'S': reply['id']['S']},
            'text': {'S': reply['text']['S']},
            'likes': {'N': reply['likes']['N']},
            'CreatedAt': { 'N': reply['CreatedAt']['N']}
        },
    )
    item = client.query(
        TableName="twitter",
        KeyConditionExpression='PK = :pk AND begins_with(SK, :follower)',
        ExpressionAttributeValues={
            ':pk': { 'S': f"USER#{parse_id_from_key(author['PK']['S'])}" },
            ':follower': { 'S': 'FOLLOWER#' }
        },
    )
    followers = item['Items']
    for follower in followers:
        response2 = client.put_item(
            TableName='twitter',
            Item={
                'PK': {'S': f"USER#{parse_id_from_key(follower['SK']['S'])}"},
                'SK': {'S': f"TIMELINE#{tweet['id']['S']}#{reply['id']['S']}"},
                'id': {'S': reply['id']['S']},
                'text': {'S': reply['text']['S']},
                'likes': {'N': reply['likes']['N']},
                'CreatedAt': { 'N': reply['CreatedAt']['N']}
            },
        )
    if response2['ResponseMetadata']['HTTPStatusCode'] == 200:
        return client.query(
            TableName="twitter",
            KeyConditionExpression='PK = :pk AND begins_with(SK, :timeline)',
            ExpressionAttributeValues={
                ':pk': { 'S': f"USER#{parse_id_from_key(author['PK']['S'])}" },
                ':timeline': { 'S': 'TIMELINE#' }
            },
        )

In [21]:
post_reply(user_martin, user_christian, tweet, reply)

{'Items': [],
 'Count': 0,
 'ScannedCount': 0,
 'ResponseMetadata': {'RequestId': 'cfd93b2a-ae22-43de-a49a-555d957702e8',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Thu, 11 Jan 2024 23:11:47 GMT',
   'x-amzn-requestid': 'cfd93b2a-ae22-43de-a49a-555d957702e8',
   'content-type': 'application/x-amz-json-1.0',
   'x-amz-crc32': '583912891',
   'content-length': '39',
   'server': 'Jetty(11.0.17)'},
  'RetryAttempts': 0}}

In [124]:
class TestPostReply(unittest.TestCase):
    def test_post_reply_enriched_replies(self):
        test_tweet = client.query(
            TableName="twitter",
            KeyConditionExpression='PK = :pk AND begins_with(SK, :timeline)',
            ExpressionAttributeValues={
                ':pk': { 'S': user_christian['PK']['S'] },
                ':timeline': { 'S': f"TIMELINE#{tweet['id']['S']}" }
            },
        )
        tweet_length = len(test_tweet['Items'])
        self.assertEqual(tweet_length, 2)
        
    def test_post_reply_empty_replies(self):
        test_tweet = client.query(
            TableName="twitter",
            KeyConditionExpression='PK = :pk AND begins_with(SK, :timeline)',
            ExpressionAttributeValues={
                ':pk': { 'S': user_christian['PK']['S'] },
                ':timeline': { 'S': f"TIMELINE#{tweet_2['id']['S']}" }
            },
        )
        tweet_length = len(test_tweet['Items'])
        self.assertEqual(tweet_length, 1)

{'Items': [{'SK': {'S': 'TIMELINE#ae6bacec-acf3-4c5b-82c3-3e80c62029db'},
   'PK': {'S': 'FOLLOWER#0c074896-cd18-4e0d-8df9-cebb9bacdfa5'},
   'text': {'S': 'Moin, moin.'},
   'tweetId': {'S': 'ae6bacec-acf3-4c5b-82c3-3e80c62029db'},
   'likes': {'N': '0'},
   'CreatedAt': {'N': '1705060848'}},
  {'SK': {'S': 'FOLLOWER#0c074896-cd18-4e0d-8df9-cebb9bacdfa5'},
   'PK': {'S': 'USER#6cca7afc-e1ec-4eca-b0eb-175480e275b5'},
   'Name': {'S': 'Christian Diegmann'}},
  {'SK': {'S': 'TWEET#ae6bacec-acf3-4c5b-82c3-3e80c62029db'},
   'PK': {'S': 'USER#6cca7afc-e1ec-4eca-b0eb-175480e275b5'},
   'id': {'S': 'ae6bacec-acf3-4c5b-82c3-3e80c62029db'},
   'text': {'S': 'Moin, moin.'},
   'likes': {'N': '0'},
   'CreatedAt': {'N': '1705060848'}},
  {'SK': {'S': 'USER#6cca7afc-e1ec-4eca-b0eb-175480e275b5'},
   'PK': {'S': 'USER#6cca7afc-e1ec-4eca-b0eb-175480e275b5'},
   'Name': {'S': 'Martin Marsal'}},
  {'SK': {'S': 'FOLLOWER#6cca7afc-e1ec-4eca-b0eb-175480e275b5'},
   'PK': {'S': 'USER#a7324fea-4a38-45a3-8

In [125]:
if __name__ == '__main__':
    suite = unittest.TestLoader().loadTestsFromTestCase(TestPostReply)
    unittest.TextTestRunner(verbosity=2).run(suite)

{'Items': [{'SK': {'S': 'TIMELINE#ae6bacec-acf3-4c5b-82c3-3e80c62029db'},
   'PK': {'S': 'FOLLOWER#0c074896-cd18-4e0d-8df9-cebb9bacdfa5'},
   'text': {'S': 'Moin, moin.'},
   'tweetId': {'S': 'ae6bacec-acf3-4c5b-82c3-3e80c62029db'},
   'likes': {'N': '0'},
   'CreatedAt': {'N': '1705060848'}},
  {'SK': {'S': 'TIMELINE#e8bdf54d-7e2d-413f-a655-5b74625ec372'},
   'PK': {'S': 'FOLLOWER#0c074896-cd18-4e0d-8df9-cebb9bacdfa5'},
   'text': {'S': 'MongoDB ist super!'},
   'tweetId': {'S': 'e8bdf54d-7e2d-413f-a655-5b74625ec372'},
   'likes': {'N': '0'},
   'CreatedAt': {'N': '1705060848'}},
  {'SK': {'S': 'FOLLOWER#0c074896-cd18-4e0d-8df9-cebb9bacdfa5'},
   'PK': {'S': 'USER#6cca7afc-e1ec-4eca-b0eb-175480e275b5'},
   'Name': {'S': 'Christian Diegmann'}},
  {'SK': {'S': 'TWEET#ae6bacec-acf3-4c5b-82c3-3e80c62029db'},
   'PK': {'S': 'USER#6cca7afc-e1ec-4eca-b0eb-175480e275b5'},
   'id': {'S': 'ae6bacec-acf3-4c5b-82c3-3e80c62029db'},
   'text': {'S': 'Moin, moin.'},
   'likes': {'N': '0'},
   'Creat

## 3rd access pattern: Edit a tweet

# 4th access pattern: Read a timeline

# 5th access pattern: Delete a Tweet

In [126]:
def delete_tweet(primaryKey, tweet_id):
    # Löschen des Tweets aus der Tweet-Entität
    client.delete_item(
        TableName='twitter',
        Key={'PK': {'S': primaryKey}, 'SK': {'S': f"TWEET#{tweet_id}"}}
    )

    # Löschen des Tweets aus den Timelines aller Follower
    followers = client.query(
        TableName="twitter",
        KeyConditionExpression='PK = :pk AND begins_with(SK, :msg)',
        ExpressionAttributeValues={
            ':pk': {'S': primaryKey},
            ':msg': {'S': 'FOLLOWER#'}
        },
    )['Items']

    for follower in followers:
        client.delete_item(
            TableName='twitter',
            Key={'PK': {'S': follower['SK']['S']}, 'SK': {'S': f"TIMELINE#{tweet_id}"}}
        )


In [127]:
def get_all_tweets():
    # Abrufen aller Tweets
    response = client.scan(
        TableName='twitter',
        FilterExpression='begins_with(SK, :msg)',
        ExpressionAttributeValues={
            ':msg': {'S': 'TWEET#'}
        }
    )

    tweets = response['Items']
    return tweets

In [128]:
# Hier rufen wir alle Tweets ab, um sie vor dem Löschen anzuzeigen
all_tweets_before_deletion = get_all_tweets()
print(f'Alle Tweets vor der Löschung: {all_tweets_before_deletion}')

Alle Tweets vor der Löschung: [{'SK': {'S': 'TWEET#ae6bacec-acf3-4c5b-82c3-3e80c62029db'}, 'PK': {'S': 'USER#6cca7afc-e1ec-4eca-b0eb-175480e275b5'}, 'id': {'S': 'ae6bacec-acf3-4c5b-82c3-3e80c62029db'}, 'text': {'S': 'Moin, moin.'}, 'likes': {'N': '0'}, 'CreatedAt': {'N': '1705060848'}}, {'SK': {'S': 'TWEET#e8bdf54d-7e2d-413f-a655-5b74625ec372'}, 'PK': {'S': 'USER#6cca7afc-e1ec-4eca-b0eb-175480e275b5'}, 'id': {'S': 'e8bdf54d-7e2d-413f-a655-5b74625ec372'}, 'text': {'S': 'MongoDB ist super!'}, 'likes': {'N': '0'}, 'CreatedAt': {'N': '1705060848'}}]


In [129]:
print(f'USER: {uuid_entry1}')
print(f'UUID: {uuid_entry5}')

USER: 6cca7afc-e1ec-4eca-b0eb-175480e275b5
UUID: e8bdf54d-7e2d-413f-a655-5b74625ec372


In [130]:
# Tweet_2 bei User Martin löschen
delete_tweet(f'USER#{uuid_entry1}', uuid_entry5)

In [131]:
# Hier rufen wir erneut alle Tweets ab, um die Änderungen anzuzeigen
all_tweets_after_deletion = get_all_tweets()
print(f'Alle Tweets nach der Löschung: {all_tweets_after_deletion}')

Alle Tweets nach der Löschung: [{'SK': {'S': 'TWEET#ae6bacec-acf3-4c5b-82c3-3e80c62029db'}, 'PK': {'S': 'USER#6cca7afc-e1ec-4eca-b0eb-175480e275b5'}, 'id': {'S': 'ae6bacec-acf3-4c5b-82c3-3e80c62029db'}, 'text': {'S': 'Moin, moin.'}, 'likes': {'N': '0'}, 'CreatedAt': {'N': '1705060848'}}]


# 6th access pattern: Delete a User

In [132]:
def delete_user(user_id):
    # Zuerst die Tweets des Benutzers abrufen
    user_tweets_response = client.query(
        TableName='twitter',
        KeyConditionExpression='#pk = :pk',
        ExpressionAttributeNames={'#pk': 'PK'},
        ExpressionAttributeValues={':pk': {'S': f'USER#{user_id}'}}
    )

    user_tweets = user_tweets_response.get('Items', [])

    # Dann jeden Tweet löschen
    for tweet in user_tweets:
        tweet_id = tweet.get('id', {}).get('S')  # Zugriff auf das tatsächlich vorhandene Feld 'id'

        if tweet_id:
            client.delete_item(
                TableName='twitter',
                Key={'PK': {'S': tweet['PK']['S']}, 'SK': {'S': tweet['SK']['S']}}
            )

            # Auch den Tweet aus den Timelines der Follower löschen
            followers_response = client.query(
                TableName='twitter',
                KeyConditionExpression='#pk = :pk',
                ExpressionAttributeNames={'#pk': 'PK'},
                ExpressionAttributeValues={':pk': {'S': tweet['PK']['S']}}
            )

            for follower in followers_response.get('Items', []):
                client.delete_item(
                    TableName='twitter',
                    Key={'PK': {'S': follower['SK']['S']}, 'SK': {'S': f'TIMELINE#{tweet_id}'}}
                )

    # Schließlich den Benutzer selbst löschen
    client.delete_item(
        TableName='twitter',
        Key={'PK': {'S': f'USER#{user_id}'}, 'SK': {'S': f'USER#{user_id}'}}
    )

In [133]:
def get_all_users():
    response = client.scan(
        TableName='twitter',
        FilterExpression='begins_with(PK, :pk) AND begins_with(SK, :sk)',
        ExpressionAttributeValues={':pk': {'S': 'USER#'}, ':sk': {'S': 'USER#'}}
    )

    users = response['Items']
    return users

In [134]:
# Alle Benutzer vor dem Löschen ausgeben
all_users_before_deletion = get_all_users()
print(f'Alle Benutzer vor dem Löschen: {all_users_before_deletion}')

Alle Benutzer vor dem Löschen: [{'SK': {'S': 'USER#6cca7afc-e1ec-4eca-b0eb-175480e275b5'}, 'PK': {'S': 'USER#6cca7afc-e1ec-4eca-b0eb-175480e275b5'}, 'Name': {'S': 'Martin Marsal'}}, {'SK': {'S': 'USER#a7324fea-4a38-45a3-82ad-27c7bf0ea941'}, 'PK': {'S': 'USER#a7324fea-4a38-45a3-82ad-27c7bf0ea941'}, 'Name': {'S': 'Robin Schüle'}}, {'SK': {'S': 'USER#0c074896-cd18-4e0d-8df9-cebb9bacdfa5'}, 'PK': {'S': 'USER#0c074896-cd18-4e0d-8df9-cebb9bacdfa5'}, 'Name': {'S': 'Christian Diegmann'}}]


In [135]:
user_christian_timeline = read_timeline(user_christian['PK']['S'])
print(user_christian_timeline)

[{'id': {'S': '36cd887d-e151-48be-a36c-2c239997d726'}, 'text': {'S': 'DynamoDB ist super'}, 'likes': {'N': '0'}, 'CreatedAt': {'N': '1705014707'}}, {'id': {'S': '5a03ab7c-daf3-4626-8fd9-00671049ca89'}, 'text': {'S': 'Moin, moin.'}, 'likes': {'N': '0'}, 'CreatedAt': {'N': '1705014707'}}, {'id': {'S': '172be746-9d26-4818-a887-35080ac97f83'}, 'text': {'S': 'Hallo zurück.'}, 'likes': {'N': '0'}, 'CreatedAt': {'N': '1705014707'}}]


In [32]:
delete_user(uuid_entry1)

In [136]:
# Alle Benutzer nach dem Löschen ausgeben
all_users_after_deletion = get_all_users()
print(f'Alle Benutzer nach dem Löschen: {all_users_after_deletion}')

Alle Benutzer nach dem Löschen: [{'SK': {'S': 'USER#a7324fea-4a38-45a3-82ad-27c7bf0ea941'}, 'PK': {'S': 'USER#a7324fea-4a38-45a3-82ad-27c7bf0ea941'}, 'Name': {'S': 'Robin Schüle'}}, {'SK': {'S': 'USER#0c074896-cd18-4e0d-8df9-cebb9bacdfa5'}, 'PK': {'S': 'USER#0c074896-cd18-4e0d-8df9-cebb9bacdfa5'}, 'Name': {'S': 'Christian Diegmann'}}]


## Channel

We create our channel and functions to save, find and count users and messages in the channel:

In [34]:
channel_town_hall = {
    'PK': { 'S': "CHAN#7" },
    'SK': { 'S': "CHAN#7" },
    'Name': { 'S': 'Town Hall' },
    'Desc': { 'S': 'General News' },
    'UserCount': { 'N': "0" },
    'MessageCount': { 'N': "0" }
}

def save_channel(channel):
    return client.put_item(
        TableName="chat",
        Item=channel
    )

# AP2
def find_channel(key):
    item = client.get_item(
        TableName="chat",
        Key={
          'PK': { 'S': key },
          'SK': { 'S': key }
        }
    )
    return item['Item'] if 'Item' in item else False

# AP4
def message_count_for_channel(channel):
    return int(channel['MessageCount']['N'])

# AP7
def user_count_for_channel(channel):
    return int(channel['UserCount']['N'])

save_channel(channel_town_hall)
channel_town_hall = find_channel(channel_town_hall['PK']['S'])
channel_town_hall

ResourceNotFoundException: An error occurred (ResourceNotFoundException) when calling the PutItem operation: Cannot do operations on a non-existent table

In [ ]:
message_count_for_channel(channel_town_hall)

In [ ]:
user_count_for_channel(channel_town_hall)

## User Join

When a user joins a channel, we create a new item and increment the user counter. Further, we setup queries to retreive the users for a channel and channels of a user:

In [ ]:
def join_channel(user, channel):
    # create userj record
    client.put_item(
        TableName="chat",
        Item={
            'PK': channel['PK'],
            'SK': { 'S': "USERJ#" + parse_id_from_key(user['PK']['S']) },
            'Name': user['Name'],
            'JoinedAt': { 'S': str(date.today()) },
            'GSI1PK': user['PK'],
            'GSI1SK': channel['PK'],
            'ChanName': channel['Name']
        }
        )
    
    # increment users
    client.update_item(
        TableName="chat",
        Key = {
            'PK': channel['PK'],
            'SK': channel['PK'],
        },
        ExpressionAttributeValues = {
            ':one': { 'N': '1' }
        },
        UpdateExpression = 'ADD UserCount :one', 
        ReturnValues = 'UPDATED_NEW'
      )

# AP5
def users_in_channel(key):
    item = client.query(
        TableName="chat",
        KeyConditionExpression='PK = :pk AND begins_with(SK, :userj)',
        ExpressionAttributeValues={
            ':pk': key,
            ':userj': { 'S': 'USERJ#' }
        }
    )
    return item['Items'] if 'Items' in item else []

# AP1
def channels_for_user(key):
    item = client.query(
        TableName="chat",
        IndexName="GSI1",
        KeyConditionExpression='GSI1PK = :pk AND begins_with(GSI1SK, :chan)',
        ExpressionAttributeValues={
            ':pk': key,
            ':chan': { 'S': 'CHAN#' }
        }
    )
    return item['Items'] if 'Items' in item else []

join_channel(user_alice, channel_town_hall)
join_channel(user_bob, channel_town_hall)
users_in_channel(channel_town_hall['PK'])

In [ ]:
channels_for_user(user_alice['PK'])

In [ ]:
find_channel(channel_town_hall['PK']['S'])

## Messages

When we send a message, we create a new item and increment the messages counter:

In [ ]:
def send_message(user, channel, text):
    timestamp = int(time.time())
    message_id = uuid.uuid4()
    
    # create message
    client.put_item(
        TableName="chat",
        Item={
            'PK': channel['PK'],
            'SK': { 'S': f"MSG#{timestamp}#{message_id}" },
            'Msg': { 'S': text },
            'CreatedAt': { 'N': str(timestamp) },
            'UserName': user['Name'],
            'UserId': user['PK']
        }
        )
    
    # increment users
    client.update_item(
        TableName="chat",
        Key = {
            'PK': channel['PK'],
            'SK': channel['PK'],
        },
        ExpressionAttributeValues = {
            ':one': { 'N': '1' }
        },
        UpdateExpression = 'ADD MessageCount :one', 
        ReturnValues = 'UPDATED_NEW'
      )

# AP3
def messages_in_channel(key, limit = 50):
    item = client.query(
        TableName="chat",
        KeyConditionExpression='PK = :pk AND begins_with(SK, :msg)',
        ExpressionAttributeValues={
            ':pk': key,
            ':msg': { 'S': 'MSG#' }
        },
        Limit = limit
    )
    return item['Items'] if 'Items' in item else []
    

send_message(user_bob, channel_town_hall, "Hey there!")
send_message(user_alice, channel_town_hall, "Hello Bob!")
send_message(user_bob, channel_town_hall, "How are you?")
messages_in_channel(channel_town_hall['PK'])

In [ ]:
find_channel(channel_town_hall['PK']['S'])